# Model as a Service


In [6]:
import pickle
import mlflow
import dagshub
from fastapi import FastAPI
from pydantic import BaseModel
from mlflow import MlflowClient


# MLflow settings
# MLFLOW_TRACKING_URI = "https://dagshub.com/zapatacc/nyc-taxi-time-prediction.mlflow"
MLFLOW_TRACKING_URI = "url/to/mlflow/track/uri"

mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_ = mlflow.search_runs(order_by=['metrics.rmse ASC'],
                          output_format="list",
                          experiment_names=["nyc-taxi-experiment-prefect"]
                          )[0]
run_id = run_.info.run_id

run_uri = f"runs:/{run_id}/preprocessor"

client.download_artifacts(
    run_id=run_id,
    path='preprocessor',
    dst_path='.'
)

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_model = mlflow.pyfunc.load_model(
    model_uri=model_uri
)


Initialized MLflow to track repo "zapatacc/nyc-taxi-time-prediction"

Repository zapatacc/nyc-taxi-time-prediction initialized!

'/Users/cristianzapata/Library/Mobile Documents/com~apple~CloudDocs/Iteso/Cursos/Proyecto en Ciencia de Datos/ProyCienDatos2024-2/Módulo 2/preprocessor'

In [ ]:
def preprocess(input_data):

    input_dict = {
        'PU_DO': input_data.PULocationID + "_" + input_data.DOLocationID,
        'trip_distance': input_data.trip_distance,
    }

    return dv.transform(input_dict)

def predict(input_data):

    X_val = preprocess(input_data)

    return champion_model.predict(X_val)

In [ ]:
app = FastAPI()

class InputData(BaseModel):
    PULocationID: str
    DOLocationID: str
    trip_distance: float


@app.post("/predict")
def predict_endpoint(input_data: InputData):
    result = predict(input_data)[0]
    return {"prediction": float(result)}

# Streamlit


In [ ]:
import streamlit as st
import requests
import json

st.write("""
# Application to predict the time for the NYC taxi trips
""")

st.sidebar.header('User Input Parameters')

def user_input_features():
    PU = st.sidebar.text_input("PU Location ID", "80")
    DO = st.sidebar.text_input("DO Location ID", "60")
    trip_distance = st.sidebar.number_input("Trip Distance", value=10, min_value=1, max_value=100)

    input_dict = {
        'PULocationID': PU,
        'DOLocationID': DO,
        'trip_distance': trip_distance,
    }

    return input_dict


input_dict = user_input_features()

### Para llamar una API desde streamlit

In [ ]:
if st.button('Predict'):
    response = requests.post(
        url="http://127.0.0.1:8000/predict",
        data=json.dumps(input_dict)
    )

    st.write(f"El tiempo estimado del viaje es: {response.json()['prediction']} minutos")